# Data wrangling

Neste notebook, performaremos Análise exploratória para encontrar padrões nos dados e determinar quais devem ser as variáveis para treinar modelos.

No dataset, existem varios casos diferentes onde o booster não obteve sucesso em pousar. Às vezes a tentativa de pouso ocorreu, mas falhou devido a um acidente; por exemplo, True Ocean significa que o rsultado do pouso foi um sucesso para uma região específica do oceano, enquanto False Ocean significa que o resultado da missão foi um fracasso no pouso para uma região específica do oceano. True RTLS significa que o resultado da missão foi um sucesso para um pad na terra, False RTLS significa que houve um fracasso no pouso na terra. True ASDS signifiac que o resultado da missão foi um sucesso para pousar em um drone marinho, False ASDS indica o fracasso

Nesta secão iremos principalmente converter esses resultados em rótulos treináveis 1 significa que o booster pousou com sucesso, 0 significa que fracassou.

# Objetivos
Performar análise exploratória de dados e determinar os rótulos de treinos

* Análise exploratória de dados
* Determinar os rótulos de treinos

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('dataset_part_1.csv')
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCSFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


In [3]:
# Identificando e calculando a porcentagem de valores faltantes em cada atributo
df.isnull().sum()/df.count()*100

FlightNumber       0.000
Date               0.000
BoosterVersion     0.000
PayloadMass        0.000
Orbit              0.000
LaunchSite         0.000
Outcome            0.000
Flights            0.000
GridFins           0.000
Reused             0.000
Legs               0.000
LandingPad        40.625
Block              0.000
ReusedCount        0.000
Serial             0.000
Longitude          0.000
Latitude           0.000
dtype: float64

In [4]:
# Identificando quais colunas são numéricas e categóricas
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

Os dados contêm várias instalações de lançamento do Space X: Cape Canaveral Space Launch Complex 40 VAFB SLC 4E, Vandenberg Air Force Base Space Launch Complex 4E (SLC-4E), Kennedy Space Center Launch Complex 39A KSC LC 39A. A localização de cada lançamento é colocada na coluna LaunchSite

Vamos ver o número de lançamentos em cada sítio.

Usando o método value_counts() na coluna LaunchSite para determinar o número de lançamentos em cada sítio:

In [5]:
# Aplica value_counts() na coluna LaunchSite
df['LaunchSite'].value_counts()

CCSFS SLC 40    55
KSC LC 39A      22
VAFB SLC 4E     13
Name: LaunchSite, dtype: int64

Cada lançamento visa uma órbita específica, e aqui estão alguns tipos de órbita comuns:

* LEO: Low Earth orbit (LEO) é uma órbita centrada na Terra com uma altitude de 2.000 km (1.200 mi) ou menos (aproximadamente um terço do raio da Terra), ou com pelo menos 11,25 períodos por dia (um período orbital de 128 minutos ou menos) e um excentricidade inferior a 0,25. A maioria dos objetos feitos pelo homem no espaço sideral estão em LEO.


* VLEO: Very Low Earth Orbits (VLEO) pode ser definido como as órbitas com altitude média inferior a 450 km. Operar nessas órbitas pode fornecer uma série de benefícios para as espaçonaves de observação da Terra, pois a espaçonave opera mais perto da observação.
* GTO: Uma órbita geossíncrona é uma órbita alta da Terra que permite que os satélites coincidam com a rotação da Terra. Localizado a 22.236 milhas (35.786 quilômetros) acima do equador da Terra, esta posição é um local valioso para monitoramento do clima, comunicações e vigilância. Como o satélite orbita na mesma velocidade que a Terra está girando, o satélite parece permanecer no mesmo lugar em uma única longitude, embora possa se deslocar de norte a sul”, escreveu a NASA em seu site do Observatório da Terra.

* SSO (or SO): É uma órbita síncrona ao Sol, também chamada de órbita heliossíncrona, é uma órbita quase polar em torno de um planeta, na qual o satélite passa sobre qualquer ponto da superfície do planeta no mesmo horário solar médio local.

* ES-L1 : Nos pontos de Lagrange, as forças gravitacionais dos dois grandes corpos se anulam de tal forma que um pequeno objeto colocado em órbita ali está em equilíbrio em relação ao centro de massa dos grandes corpos. L1 é um desses pontos entre o sol e a terra.

* HEO Uma órbita altamente elíptica, é uma órbita elíptica com alta excentricidade, geralmente referindo-se a uma em torno da Terra.

* ISS Uma estação espacial modular (satélite artificial habitável) em órbita baixa da Terra. É um projeto colaborativo multinacional entre cinco agências espaciais participantes: NASA (Estados Unidos), Roscosmos (Rússia), JAXA (Japão), ESA (Europa) e CSA (Canadá).

* MEO Órbitas geocêntricas variando em altitude de 2.000 km (1.200 milhas) até um pouco abaixo da órbita geossíncrona a 35.786 quilômetros (22.236 milhas). Também conhecida como órbita circular intermediária. Estes são "mais comumente em 20.200 quilômetros (12.600 milhas), ou 20.650 quilômetros (12.830 milhas), com um período orbital de 12 horas

* HEO Órbitas geocêntricas acima da altitude da órbita geossíncrona (35.786 km ou 22.236 mi)

* GEO É uma órbita geossíncrona circular 35.786 quilômetros (22.236 milhas) acima do equador da Terra e seguindo a direção da rotação da Terra

* PO É um tipo de satélite em que um satélite passa acima ou quase acima de ambos os pólos do corpo que está sendo orbitado (geralmente um planeta como a Terra)

Usando o método .value_counts() para determinar o número de ocorrências em cada órbita.

In [6]:
df.Orbit.value_counts()

GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
ES-L1     1
HEO       1
SO        1
GEO       1
Name: Orbit, dtype: int64

Usando o método .value_counts() na coluna outcome para determinar o número de resultados. Então assinalando a uma variável landing_outcomes

In [7]:
landing_outcomes = df.Outcome.value_counts()

In [8]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


In [9]:
# Criamos um conjunto de resultados em que o primeiro estágio não pousou adequadamente 

bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

Usando a coluna Outcome, criamos uma lista onde o elemento é zero se a correspondente linha em Outcome está no conjunto bad_outcome; caso contrário, será 1. então assinalamos a variável landing_class

In [10]:
# landing_class = 0 se está em bad_outcome
# landing_class = 1 caso contrário
landing_class = list()
for out in df.Outcome:
    if out in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)

Esta variável representará a variável de classificação que representa o resultado de cada lançamento. Se o valor for zero, o primeiro estágio não pousou com sucesso; um significa que o primeiro estágio pousou com sucesso

In [11]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [12]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


In [13]:
# Podemos usar a seguinte linha de código para verificar a taxa de sucesso no pouso:

df["Class"].mean()

0.6666666666666666

In [14]:
df.to_csv("dataset_part_2.csv", index=False)